##  The Grover Algorithm: Implementation


_course: quantum cryptography for beginners
<br>date: 30 november 2022
<br>author: burton rosenberg_



### The  Walsh Hadamard Transformation

To implement the Grover Algorithm we need to implement two refections. The oracle reflection:

$$
U_f:\;|\phi\rangle \longrightarrow \sum_i (-1)^{(i==j)}\,|i\rangle
$$

and the flip around the average,

$$
    U_{r'} = 2 |h_0\rangle\langle h_0| - I
$$

Where $h_0$ is the result of putting zero through a bank of Hadamard transfomations,

$$
  h_0 = H^{n} |0\rangle
$$

and in general the Walsh-Hadamard basis is,

$$
 h_i = H^{n} |i\rangle
$$

whichi is a basis as the transformation is an isometry and the i's are a basis. The vectors in this basis are remarkably a pattern of +1's and -1's that described as,

$$
H^{n} |i\rangle = \sum_j (-1)^{i\cdot j}\,|i\rangle
$$

where you are to interpret $i\cdot j$ as the inner product in $\mathcal{Z}_2{}^n$ having expressed $i, j\in \mathcal{Z}_2{}^n$ by looking at their binary expansion. For instance,


|  i | +/-j |
| ---- | ---- |
| 0 | + + + + |
| 1 | + - + - |
| 2 | + + - - |
| 3 | + - - + |

In fact, we do not implement the reflection around $h_0$ rather around it perpendicular in the plane containing it and the vector $|j\rangle$ representing the unknown. This upsets our plan by a minus sign, as the two reflections differ by a half rotation $x\rightarrow -x$,

\begin{eqnarray}
U_r &=& -U_{r'} \\
&=& I - 2 |h_0\rangle\langle h_0|  \\
&=& \sum_i |h_i\rangle \langle h_i | - 2 |h_0\rangle\langle h_0| \\
&=& \sum_i (-1)^{i==0}\, |h_i\rangle \langle h_i | 
\end{eqnarray}

The final element is how to reflect around $|i\rangle$, expressed as,

$$
U_i  = \sum_j (-1)^{i==j}\,\,|j\rangle
$$

This is done with a controlled Z gate, applied to any qubit and controlled by all the remaining qubits. 

To reflect around $|0\rangle$, negate with an X all but one of the qubits, use Tofolli gates and ancillas as needed to logically and (classical) all the qubits, and use the result as the control to a controlled Z on the uninverted qubit. Then uncompute the anciliaries and the X negations.

##### Example

In [27]:
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import numpy as np
import math
import cmath
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

#Aer.backends()

def Ur():
    qc = QuantumCircuit(4)
    for i in range(3):
        qc.h(i)
        qc.x(i)
    qc.ccx(0,1,3)
    qc.cz(3,2)
    qc.ccx(0,1,3)
    for i in range(3):
        qc.x(i)
        qc.h(i)
    return qc

print(Ur())

     ┌───┐┌───┐             ┌───┐┌───┐
q_0: ┤ H ├┤ X ├──■───────■──┤ X ├┤ H ├
     ├───┤├───┤  │       │  ├───┤├───┤
q_1: ┤ H ├┤ X ├──■───────■──┤ X ├┤ H ├
     ├───┤├───┤  │       │  ├───┤├───┤
q_2: ┤ H ├┤ X ├──┼───■───┼──┤ X ├┤ H ├
     └───┘└───┘┌─┴─┐ │ ┌─┴─┐└───┘└───┘
q_3: ──────────┤ X ├─■─┤ X ├──────────
               └───┘   └───┘          



### The Grover's Algorithm on a GPU

It might be good to refelction on what is the quantum advantage of Grover's algorithm. To do so we can implement it non-quantum. The Walsh-Hadamard transform can be done on a GPU with log n time complexity. Since the fan-in to the controled Z will require a log n tree of ands, the time complexity will match that of the quantum algorithm.

However, we will need an exponential number of GPU threads. While $n$ qubits can encode $2^n$ complex numbers, we will need $2^n$ GPU threads, one for each complex number. However, there are about 100,000 GPU threads on some Nvidia cards. It might be the bottle neck of memory access with is truely the problem with Grovers done on a classical computer.

See the wikipedia description of the [Fast Walsh Hadamard](https://en.wikipedia.org/wiki/Fast_Walsh%E2%80%93Hadamard_transform).


In [26]:

import numpy as np
import math

class Grover_GPU:
    
    def __init__(self,k):
        self.iter = int(math.sqrt(2**k))
        self.phi = np.ones(k)/math.sqrt(k)
        pass
    
    @staticmethod
    def wh_transform(a):
        # assume len(a) = 2^i
        
        def wh_kernel(a,i,d):
            if i&d==0: a[i], a[i+d] = a[i]+a[i+d], a[i]-a[i+d]
            return 

        n = len(a)    
        d = n//2
        while True:
            # parrallel kernel launches
            for i in range(n): wh_kernel(a,i,d)
            if d==1: break
            d = d//2

    def reflect_h0(self):
        self.wh_transform(self.phi)
        self.phi[0] = -self.phi[0]
        self.wh_transform(self.phi)
        self.phi = self.phi/len(self.phi)
        return

    def reflect_j(self,j):
        self.phi[j] = -self.phi[j]
        return

    def grover_step(self,j):
        self.reflect_j(j)
        self.reflect_h0()
        return
    
    def observation(self,j):
        d = self.phi[j]
        return d*d

    def coords(self,j):
        r = self.phi[j]*self.phi[j]
        if r>1.0:
            r = 1.0
        return (math.sqrt(r),math.sqrt(1-r))

g = Grover_GPU(128)
j = 4
for i in range(16):
    g.grover_step(j)
    print(f'obs: {g.observation(j):.4f}')
    

obs: 0.0689
obs: 0.1834
obs: 0.3372
obs: 0.5111
obs: 0.6837
obs: 0.8335
obs: 0.9420
obs: 0.9956
obs: 0.9878
obs: 0.9194
obs: 0.7991
obs: 0.6416
obs: 0.4666
obs: 0.2957
obs: 0.1502
obs: 0.0480


### Exercise A

Use statevectors to test that refection around the average circuit given above.

### Exercise B

Complete the 3 qubit Grovers circuit and test.

### Excercise C

Create a python function that builds a k qubit Grovers circuit.


## Answers C

In [29]:
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import numpy as np
import math
import cmath
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

class Grover:

    def __init__(self,n,s,barriers=False):
        assert n>3
        assert s<2**n
        
        self.n = n
        self.k = n - 2  # number of anciliaries needed
        self.qc = QuantumCircuit(n+self.k)
        
        self.hadamard()
        if barriers: 
            self.qc.barrier()
        self.reflect_around_j(s)
        if barriers:
            self.qc.barrier()
        self.reflect_around_plus()
        self.qc.measure_all()
        
    def cnz(self):
        assert self.n>=3
        a = self.n
        self.qc.ccx(0,1,self.n)
        for i in range(2,self.n-1):
            self.qc.ccx(i,a,a+1)
            a += 1
        self.qc.cz(self.n-1,a)
        for i in range(self.n-2,1,-1):
            a -= 1
            self.qc.ccx(i,a,a+1)
        self.qc.ccx(0,1,self.n)
        return

    def reflect_around_plus(self):
        for i in range(self.n):
            self.qc.h(i)
            self.qc.x(i)
        self.cnz()
        for i in range(n):
            self.qc.x(i)
            self.qc.h(i)
        return

    def reflect_around_j(self,j):
        self.select_j(j)
        self.cnz()
        self.select_j(j)
        return

    def select_j(self,j):
        for i in range(self.n):
            if j%2==0:
                self.qc.x(i)
            j = j//2
        return

    def hadamard(self):
        for i in range(self.n):
            self.qc.h(i)
        return
    
    def draw(self):
        print(self.qc.draw())
        
    def run(self,how_many=12):
        simulator = Aer.get_backend('aer_simulator')
        qc = transpile(self.qc,simulator)
        result = simulator.run(qc).result()
        counts = result.get_counts(qc)
        x = { k: counts[k] for k in sorted(counts,key=counts.get,reverse=True)}
        print(x)
                

n = 6 # qubits
s = 4 # secret number

grover = Grover(n,s,barriers=True)
grover.draw()
grover.run()


         ┌───┐ ░ ┌───┐                                                     »
    q_0: ┤ H ├─░─┤ X ├──■───────────────────────────────────────────────■──»
         ├───┤ ░ ├───┤  │                                               │  »
    q_1: ┤ H ├─░─┤ X ├──■───────────────────────────────────────────────■──»
         ├───┤ ░ └───┘  │                                               │  »
    q_2: ┤ H ├─░────────┼────■────────────────────────────────■─────────┼──»
         ├───┤ ░ ┌───┐  │    │                                │  ┌───┐  │  »
    q_3: ┤ H ├─░─┤ X ├──┼────┼────■──────────────────────■────┼──┤ X ├──┼──»
         ├───┤ ░ ├───┤  │    │    │               ┌───┐  │    │  └───┘  │  »
    q_4: ┤ H ├─░─┤ X ├──┼────┼────┼────■───────■──┤ X ├──┼────┼─────────┼──»
         ├───┤ ░ ├───┤  │    │    │    │       │  ├───┤  │    │         │  »
    q_5: ┤ H ├─░─┤ X ├──┼────┼────┼────┼───■───┼──┤ X ├──┼────┼─────────┼──»
         └───┘ ░ └───┘┌─┴─┐  │    │    │   │   │  └───┘  │    │       ┌─┴─┐»